In [3]:
import datetime
Startdate=datetime.datetime.now().date()-datetime.timedelta(30)
finishDate=datetime.datetime.now().date()-datetime.timedelta(1)

In [5]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
# Query to get the maximum date
max_date_query = """
    SELECT DATEADD(day, 1, MAX([Date])) as MaxDate
    FROM [DataforAnalysis].[dbo].[Store_SPC];
"""
max_date_df = pd.read_sql(max_date_query, cnxn)

# Printing the maximum date with one day added
max_date_plus_1 = max_date_df['MaxDate'][0]
print(f"Maximum Date + 1 day: {max_date_plus_1}")

query = f"""
EXEC  [DataforAnalysis].[dbo].[FlyHeight] '{max_date_plus_1}','{finishDate}','FlyHeight'
        """
datasets = pd.read_sql(query, cnxn)
print(datasets.head(100000))

Maximum Date + 1 day: 2023-08-29


C:\Users\IT\AppData\Local\Temp\ipykernel_9380\1606050750.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  max_date_df = pd.read_sql(max_date_query, cnxn)
C:\Users\IT\AppData\Local\Temp\ipykernel_9380\1606050750.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, cnxn)


TypeError: 'NoneType' object is not iterable

In [31]:
# Your existing code here...

# Execute the SQL query and store the results in the 'datasets' DataFrame
datasets = pd.read_sql(query, cnxn)

# Your existing code here...

# Establish a new connection for inserting the data into the target table
insert_server = '192.168.101.219'
insert_database = 'DataforAnalysis'
insert_username = 'DATALYZER'
insert_password = 'NMB54321'
insert_cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+insert_server+';DATABASE='+insert_database+';UID='+insert_username+';PWD='+ insert_password)
insert_cursor = insert_cnxn.cursor()

# Iterate through the rows of the 'datasets' DataFrame and insert data into the target table
for index, row in datasets.iterrows():
    insert_query = f"""
        INSERT INTO [DataforAnalysis].[dbo].[Store_SPC] ([Date], [Model], [Line], [Machine], [%Yield], [Cycle_time (sec)],
        [Down time (min)], [AVG], [STD], [CPK], [Parameter])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    values = (row['Date'], row['Model'], row['Line'], row['Machine'], row['%Yield'], row['Cycle_time (sec)'],
              row['Down time (min)'], row['AVG'], row['STD'], row['CPK'], row['Parameter'])
    insert_cursor.execute(insert_query, values)

# Commit the changes and close the connection
insert_cursor.commit()
insert_cursor.close()
insert_cnxn.close()


C:\Users\IT\AppData\Local\Temp\ipykernel_17720\3032691257.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, cnxn)


In [32]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
# Query to get the maximum date
max_date_query = """
    SELECT DATEADD(day, 1, MAX([Date])) as MaxDate
    FROM [DataforAnalysis].[dbo].[SPC_STD_MC];
"""
max_date_df = pd.read_sql(max_date_query, cnxn)

# Printing the maximum date with one day added
max_date_plus_1 = max_date_df['MaxDate'][0]
print(f"Maximum Date + 1 day: {max_date_plus_1}")

query = f"""
  DECLARE @DynamicSQL AS NVARCHAR(max),
 @Start NVARCHAR(30) = '{max_date_plus_1}',
 @End NVARCHAR(30) = '{finishDate}',
 @Parameter NVARCHAR(30) = 'Parallelism';

    WITH BarcodeThan100 as(
      select [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] as [Date]
             ,count([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Barcode]) as  [countBarcode]
             from [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester]
                INNER JOIN [TransportData].[dbo].[Master_matchings]
                ON [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Model = [TransportData].[dbo].[Master_matchings].Model
             where [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] BETWEEN  @Start AND  @End
             and [TransportData].[dbo].[Master_matchings].[createdAt] = (select max([TransportData].[dbo].[Master_matchings].[createdAt]) from [TransportData].[dbo].[Master_matchings])
       group by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date]
       )
    ,XT (x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11) as
        (select [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] as [Date]
        ,case when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) < 120
        then datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time))
        when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) >= 120 then 0 end as [C/T in sec]
        ,case when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) >= 120
        then datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time))
        when datediff(SECOND,0,cast((lag([DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) over (order by [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time] desc) - [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Time]) as time)) < 120 then 0 end as [D/T]
        ,[DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Parallelism as [Parameter]
        ,[DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Model] as [model]
        ,[TransportData].[dbo].[Master_matchings].LSL as [LSL]
        ,[TransportData].[dbo].[Master_matchings].CL as [CL]
        ,[TransportData].[dbo].[Master_matchings].USL as [USL]
        ,'Average' as [AVG]
        ,[DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Barcode]
		,Line
        from BarcodeThan100
                                left join [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester]
                                on BarcodeThan100.Date = [Dynamic_Parallelism_Tester].Date
        INNER JOIN [TransportData].[dbo].[Master_matchings]
        ON [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].Model = [TransportData].[dbo].[Master_matchings].Model
         where [DataforAnalysis].[dbo].[Dynamic_Parallelism_Tester].[Date] BETWEEN  @Start AND  @End
        and [TransportData].[dbo].[Master_matchings].[Parameter] = @Parameter
        and [TransportData].[dbo].[Master_matchings].[createdAt] = (select max([TransportData].[dbo].[Master_matchings].[createdAt]) from [TransportData].[dbo].[Master_matchings])
        and countBarcode >100
        and Operator = 'Operator'
        ),
      XO (x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13) as(
      select x1 as [Date]
      ,cast(convert(float,count(case when x4 >= x6 and x4 <= x8 then 1 else null end))/
      convert(float,count(case when x4 >= x6 and x4 <= x8 then 1 else null end)
      + count(case when x4 < x6 or x4 > x8 then 1 else null end)) * 100 AS DECIMAL(10, 2) ) as [%Yield]
      ,cast(convert(float,sum(x2))/count(x2) as DECIMAL(10, 2)) as [Cycle_time (sec)]
      ,cast(convert(float,sum(x3))/60 as decimal(10,0)) as [Down time (min)]
      ,cast(AVG(x4) as decimal(10,3)) as [AVG]
      ,cast(stdev(x4) as decimal(10,4)) as [STD]
      ,case when (cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)) < cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))) or
      ((cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)) > cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))) and x6 = 0)
      then cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)) when cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2))
      > cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))
      then cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2)) when cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2))
      = cast(convert(float,((AVG(x4)-x6)/(3*STDEV(x4)))) as decimal(10,2))
      then cast(convert(float,((x8-AVG(x4))/(3*STDEV(x4)))) as decimal(10,2)) end as [CPK]
      ,x5 as [Model]
      ,x6 as [LSL]
      ,x7 as [CL]
      ,x8 as [USL]
      ,x9 as [Machine]
	  ,x11 as Line
      from XT
      group by x1,x5,x6,x7,x8,x9,x11)



      select 
      XO.x1 as [Date]
	  ,XO.x8 as [Model]
      ,XO.x13 as Line
      ,XO.x12 as [Machine]
      ,XO.x2 as [%Yield]
      ,XO.x3 as [Cycle_time (sec)]
      ,XO.x4 as [Down time (min)]
      ,XO.x5 as [AVG]
      ,XO.x6 as [STD]
      ,XO.x7 as [CPK]
    
      ,x11 as [USL]
      ,x10 as [CL]
      ,x9 as [LSL]
	  ,@Parameter as Parameter
      from XO
        """
dataSTD = pd.read_sql(query, cnxn)
print(dataSTD.head(100000))

Maximum Date + 1 day: 2023-08-29
Empty DataFrame
Columns: [Date, Model, Line, Machine, %Yield, Cycle_time (sec), Down time (min), AVG, STD, CPK, USL, CL, LSL, Parameter]
Index: []


C:\Users\IT\AppData\Local\Temp\ipykernel_17720\613617377.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  max_date_df = pd.read_sql(max_date_query, cnxn)
C:\Users\IT\AppData\Local\Temp\ipykernel_17720\613617377.py:112: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataSTD = pd.read_sql(query, cnxn)


In [33]:
# Your existing code here...

# Execute the SQL query and store the results in the 'datasets' DataFrame
datasets = pd.read_sql(query, cnxn)

# Your existing code here...

# Establish a new connection for inserting the data into the target table
insert_server = '192.168.101.219'
insert_database = 'DataforAnalysis'
insert_username = 'DATALYZER'
insert_password = 'NMB54321'
insert_cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+insert_server+';DATABASE='+insert_database+';UID='+insert_username+';PWD='+ insert_password)
insert_cursor = insert_cnxn.cursor()

# Iterate through the rows of the 'datasets' DataFrame and insert data into the target table
for index, row in dataSTD.iterrows():
    insert_query = f"""
        insert into [DataforAnalysis].[dbo].[SPC_STD_MC] ([Date], [Model], [Line], [Machine], [%Yield], [Cycle_time (sec)],
        [Down time (min)], [AVG], [STD], [CPK],[LSL],[CL],[USL],[Parameter])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? , ? )
    """
    values = (row['Date'], row['Model'], row['Line'], row['Machine'], row['%Yield'], row['Cycle_time (sec)'],
              row['Down time (min)'], row['AVG'], row['STD'], row['CPK'],row['LSL'],row['CL'],row['USL'], row['Parameter'])
    insert_cursor.execute(insert_query, values)

# Commit the changes and close the connection
insert_cursor.commit()
insert_cursor.close()
insert_cnxn.close()

C:\Users\IT\AppData\Local\Temp\ipykernel_17720\4262995815.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, cnxn)
